* This version has within 2sigma ACC everywhere but is noisier

In [1]:
import subprocess
import numpy as np
import healpy as hp
import pickle
from random import randint
import joblib
import sklearn
import sys
import importlib

sys.path.append('../methods_code_Nresol/')
sys.path.append('../utils/')
from utils_circpatch import *
from methods2 import *
from vis_utils import *
import astro_cuts
importlib.reload(astro_cuts)

<module 'astro_cuts' from '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/astro_cuts.py'>

In [2]:
fullskypatch = get_testbed_dict('FullSky_Bayestar_babsgt20')

In [3]:
numtiles = np.unique(get_largepix_for_smallpix(fullskypatch['coords'], 2048, 32))

In [4]:
len(numtiles), len(numtiles)/2368

(6620, 2.795608108108108)

In [5]:
b17map = get_bayestar2017_map()
np.isnan(b17map[fullskypatch['coords']]).sum()

0

In [6]:
runname = 'fullsky_runs/15e_fwhm-15_'
wisemodel = joblib.load('spectra_matched/models/svm_panstarrs_quasarsep_balanced.joblib')
smodel = joblib.load('spectra_matched/models/svm_sdss_quasarsep_unbalanced.joblib')

cuts_list = [('distmod_median_cut', {'mindm': 8}), ('e_nonneg_cut_sigfac', {'sigfac': 5.0}),
            ('bayestar_chisq_per_passband', {'maxchisq': 5.0}), 
            ('wise_svmnondetectioncombinedcut', {'model': wisemodel}), ('sdss_uvcut', {'model': smodel}),
            ('dm_sigma_cut', {'maxsig': 1.5})]

outer_kwargs = {'Nsidetile': 32, 'radius_deg_extra': 35/60, 'Nsideresol': 2048,
               'stars_presaved': '/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits'}
recon_kwargs = {'length': np.deg2rad(15/(2.355*60)), 'scale': 1.0, 'no_invvar': False}
kwargs_dict = {'outer_kwargs': outer_kwargs, 'recon_kwargs': recon_kwargs}
recon_func_name = 'gaussiankernel_reconstruction_effective_spherical3_blocks'

In [ ]:
recondict = gnupartilewise_new(fullskypatch['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=7500, mem=8000, compile_pickle=True)

Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits


In [11]:
missingtiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missingtiles.append(tile)

In [14]:
recondict = gnupartilewise_new(fullskypatch['coords'], recon_func_name, cuts_list, kwargs_dict, 
                               runname, sleep=1500, mem=16000, compile_pickle=True)

Dir already exists
Tiles file already exists
Only reconstructing for missing 142 tiles
Using Star Directory: /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Tmp pickle already exists
Ckp
Missing / Corrupted files:  [8131]


TypeError: cannot unpack non-iterable NoneType object

In [15]:
missingtiles = []
for tile in numtiles:
    if not os.path.exists(runname+'_tmpdir/recon_{}.hdf5'.format(tile)):
        missingtiles.append(tile)

In [16]:
missingtiles

[8131]

In [17]:
missing_coords = []
for t32 in missingtiles:
    missing_coords.append(get_smallpix_in_tilepix(outer_kwargs['Nsidetile'], t32, 2048))
missing_coords = np.hstack(missing_coords)

In [18]:
len(missing_coords)

4096

In [19]:
looptilewise(missing_coords, recon_func_name, cuts_list, kwargs_dict, runname, save_tilewise=True, compile_pickle=False)

Looping over 1 tiles
Using dir  /n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/data/lsdraw/stars_edr3/{}.fits
Region for tile8131


/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:44: RuntimeWarning: divide by zero encountered in true_divide
  df['reduced_chisq'] = df['chisq'].to_numpy() * n_passbands / (n_passbands - 4)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:49: RuntimeWarning: divide by zero encountered in power
  sdss_flux_sig = np.power(np.array(dat['sdss_dr14_starsweep.psfflux_ivar']), -0.5)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:51: RuntimeWarning: divide by zero encountered in log10
  df['sdss.pmag_'+b] = 22.5 - 2.5*np.clip(np.log10(np.array(dat['sdss_dr14_starsweep.psfflux'])[:, ib]), 0.0, np.inf)
/n/holylfs05/LABS/finkbeiner_lab/Everyone/highlat/notebooks_main1/../methods_code_Nresol/do_recon_tilewise.py:56: RuntimeWarning: divide by zero encountered in log10
  df['ps.psfmagmean_'+b] = -2.5*np.log10(np.arr

Cut0 598227 0.9954257740741727
Cut1 596087 0.9918648997630521
Cut2 598048 0.995127925241607
Cut3 589906 0.9815799632597642
Cut4 600976 1.0
Cut5 590399 0.9824002955192886
Cuts_All 570588 0.9494355847820878
Tile 8131, NumPixels=4096, NumStars=570588, Star/Pix ratio = 139.3037109375
SigRef 0.19747676285489194
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 1204, 16%ile = 1302.0, median stars = 1372.0, 84%ile = 1454.0, max stars = 1594
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 1242, 16%ile = 1430.0, median stars = 1496.0, 84%ile = 1555.0, max stars = 1641
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 1110, 16%ile = 1221.0, median stars = 1373.0, 84%ile = 1468.0, max stars = 1743
No stars for pix 0.0
Stars Contributing >1sigma/N(pixel) of the weight matrix: Min stars = 1357, 16%ile = 1559.0, median stars = 1674.0, 84%ile = 1769.0, max stars = 1846


In [20]:
reconmap, varmap, reconpix = get_Nsideresol_healpix_map_from_gnuparpatches_safe(runname+'_tmpdir/', Nsideresol=kwargs_dict['outer_kwargs']['Nsideresol'], tiles = numtiles)
result_dict = {'dustmap': reconmap, 'variancemap': varmap, 'reconpix': reconpix,
               'func': 'gnupartilewise_new',
               'cuts_list': cuts_list, 'recon_func_name': recon_func_name,
               'kwargs_dict': kwargs_dict}
pickle.dump(result_dict, open(runname + '_babsgt20.pkl', 'wb'))

Ckp
